In [ ]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-0fac709e-65a6-e992-73d1-99b88ca71668)


In [ ]:
import zipfile

def unzip_data(filename):
  
  zip_ref = zipfile.ZipFile(filename, "r")
  zip_ref.extractall()
  zip_ref.close()

In [ ]:

!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"


unzip_data("nlp_getting_started.zip")

--2022-01-09 16:03:27--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.201.128, 74.125.202.128, 173.194.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.201.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2022-01-09 16:03:27 (111 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [ ]:
from sklearn.model_selection import train_test_split


train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state = 42)
                                                           

In [ ]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [ ]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation", 
                                    split="whitespace",
                                    ngrams=None, 
                                    output_mode="int", 
                                    output_sequence_length=None) 
                                    

In [ ]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [ ]:

max_vocab_length = 10000 
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [ ]:
text_vectorizer.adapt(train_sentences)

In [ ]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
import random
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
@Reddakushgodd she said a few months. But I get a curfew for out time smfh      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[9276,  191,  205,    3,  548, 1119,   30,    8,   52,    3,  378,
          10,   36,   92, 8379]])>

In [ ]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] 
bottom_5_words = words_in_vocab[-5:] 
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [ ]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, 
                             output_dim=128, 
                             embeddings_initializer="uniform", 
                             input_length=max_length, 
                             name="embedding_1") 

embedding

In [ ]:

random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")


sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
So a train derailed and instead of me getting to work early like I would've I'm now late as fuck      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02943667,  0.0354729 ,  0.00599856, ..., -0.02309817,
          0.03555591, -0.04938353],
        [-0.04284013, -0.01489798, -0.0159496 , ..., -0.01166106,
          0.03061062,  0.01972148],
        [ 0.01869095, -0.00441112,  0.00571845, ..., -0.04840523,
         -0.00107772,  0.04899628],
        ...,
        [-0.03071752,  0.04972876, -0.01677845, ..., -0.03565687,
          0.0019665 , -0.04666593],
        [ 0.03494309,  0.02677599, -0.01469069, ..., -0.00333368,
         -0.02596296, -0.02523291],
        [ 0.0466082 ,  0.04171934,  0.00925603, ..., -0.04540265,
          0.00479609,  0.0305157 ]]], dtype=float32)>

In [ ]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.02943667,  0.0354729 ,  0.00599856,  0.03872592, -0.03130857,
       -0.01039084, -0.01581337,  0.04074533,  0.04188856, -0.00504326,
       -0.02969487, -0.00721793,  0.02162489,  0.02305296, -0.03500966,
       -0.02379768, -0.01612772,  0.03151325, -0.03999344, -0.04900312,
       -0.01138946, -0.0217517 ,  0.04429324,  0.03507403,  0.00847769,
        0.0194279 ,  0.01666734,  0.0210503 ,  0.03949357,  0.01715526,
       -0.04528803, -0.00695497, -0.03550465, -0.04060518,  0.01009575,
       -0.00250943,  0.03137875, -0.0446262 , -0.02981209, -0.03842891,
       -0.00140982,  0.00177037,  0.01546688, -0.01057224,  0.03884915,
        0.03152663,  0.00733941,  0.04938952, -0.00885748, -0.03655487,
       -0.01069647,  0.00602775,  0.04059019,  0.03608254, -0.04321803,
        0.03578779,  0.00183564,  0.00134137,  0.01545522,  0.0418924 ,
        0.02862312,  0.04945599, -0.03931342, -0.03070066,  0.00877895,
        0.041324

In [ ]:

tf.random.set_seed(42)
from tensorflow.keras import layers
model_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding")



inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_embedding(x)
print(x.shape)
x = layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = layers.LSTM(64)(x) # return vector for whole sequence
print(x.shape)
x = layers.Dense(64, activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs, name="model_LSTM")

(None, 15, 128)
(None, 64)


In [ ]:
model.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model.summary()

Model: "model_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                        

In [ ]:
# Fit model
model_history = model.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 12s 20ms/step - loss: 0.5105 - accuracy: 0.7475 - val_loss: 0.4579 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3205 - accuracy: 0.8704 - val_loss: 0.4995 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.2233 - accuracy: 0.9143 - val_loss: 0.5521 - val_accuracy: 0.7559
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1623 - accuracy: 0.9434 - val_loss: 0.6322 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1130 - accuracy: 0.9588 - val_loss: 0.9048 - val_accuracy: 0.7651


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  
  
  model_accuracy = accuracy_score(y_true, y_pred) * 100
 
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [ ]:
model_pred_probs = model.predict(val_sentences)
model_pred_probs.shape, model_pred_probs[:10]

((762, 1), array([[9.8280832e-03],
        [6.8554831e-01],
        [9.9961996e-01],
        [2.5565855e-02],
        [7.3812174e-04],
        [9.9977916e-01],
        [9.1003805e-01],
        [9.9987817e-01],
        [9.9978107e-01],
        [1.5058218e-01]], dtype=float32))

In [ ]:
model_preds = tf.squeeze(tf.round(model_pred_probs))
model_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [ ]:
model_results = calculate_results(y_true=val_labels,
                                    y_pred=model_preds)
model_results

{'accuracy': 76.50918635170603,
 'f1': 0.7624764882687336,
 'precision': 0.7675147482403992,
 'recall': 0.7650918635170604}